# Homework 4

In [ ]:
import py2neo

graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "monneo4j"), name="movie.db")
#graph.delete_all()
# 1) how many actors played in more than 2 movies? Get their nales and year they were born;

query="""match (p:Person)-[:ACTED_IN]->(q:Movie) with p,
count(q) as view  where view >2
return p,view
"""
graph.run(query)
# 2) In how many movies did Tom Hanks play?
query=""" match (p:Person{name: "Tom Hanks"})-[:ACTED_IN]->(q:Movie) with count(q) as nbr return nbr"""
graph.run(query)

# 3) In average how many movies do actors play in ?
query="""match (p:Person)-[:ACTED_IN]->(q:Movie) with p,count(q) as view return avg(view)"""
graph.run(query)

# 4) In average how many actors are there in a movie?
query="""match (pp:Person)-[:ACTED_IN]->(q:Movie) with q,count(pp) as nbr return avg(nbr)"""
graph.run(query)

# 5) In average how many writers is needed to write a movie ?
query="""match (pp:Person)-[:WROTE]->(q:Movie) with q,count(pp) as nbr return avg(nbr)"""
graph.run(query)

# 6) What is the proportion of writers that also directed the movie ?
query="""match (p:Person)-[:DIRECTED]->(m:Movie)<-[:WROTE]-(k:Person)
where p.name= k.name
RETURN (p.name) as kad, m.title as movie"""
graph.run(query)

# 7) Do an histogram of the year of release for the movies.

import seaborn as sns
%matplotlib inline

query ="""match (année:Movie) return année.released """
data=graph.run(query).to_data_frame()
sns.distplot(data['année.released'])

sns.displot(data=data, x="année.released")

# 8) Who are the 10 actors with the most movies?
query="""match (pp:Person)-[ACTED_IN]->(q:Movie) return pp.name,
count(q) as ka order by ka desc limit 10"""
graph.run(query)

# 9) Delete every movie made by a director of your choosing
query="""match (pp:Person{name:"james Marshall"})-[ACTED_IN]->(q:Movie) detach delete q"""
graph.run (query)

# 10) Give some statistics that you find interesting.

# some statistics about the popularity of the actors

query5= """MATCH (a:person) 
WHERE a.movie = 7
RETURN a.movie AS movie, a.name AS Name 
"""
datapop = graph.run(query5).to_data_frame()
# datapop donne les 10 acteurs célèbres avec le nombre de movies>7

# Homework 5


In [ ]:
# 1) Give the IDs of the top 10 streamers that have atleast one collaboration.

import pandas as pd
import tqdm
import py2neo

import tqdm
import itertools
# initial creation import py2neo

target = pd.read_csv("data/Chap4/Homework_data/Twitch/FR_target.csv")
graph = py2neo.Graph("bolt://localhost:7687", auth=("neo4j", "monneo4j"), name="movie.db")
#graph.delete_all()

try:
    graph.run("CREATE CONSTRAINT ON (n:Streamer) ASSERT n.id_ IS UNIQUE")
except:
    print("Constraint already exists")
  
transaction_list = []

iteration = 0
for row in tqdm.tqdm(target.iterrows()):
    id_ = row[1]["id"]
    views = row[1]["views"] 
    days = row[1]["days"]
    mature = row[1]["mature"] 
    new_id = row[1]["new_id"]
    partner = row[1]["partner"] 
    transaction_list.append({"id_":id_,"views":views,"days":days,"mature":mature,"new_id":new_id,"partner":partner})
    iteration += 1
    if iteration % 1000 == 0:
        transaction = "UNWIND $json as data CREATE (n:Streamer) SET n = data"
        graph.run(transaction, json=transaction_list)
        transaction_list = []
        
graph.run(transaction, json=transaction_list)

In [ ]:
edges = pd.read_csv("data/Chap4/Homework_data/Twitch/FR_edges.csv")

transaction_list = []


transaction = """UNWIND $json as data
MATCH (a:Streamer),(b:Streamer)
WHERE a.new_id = data.from AND b.new_id = data.to
MERGE (a)-[r:RELATION]->(b)
ON CREATE
    SET r.n_collab = 1
ON MATCH
    SET r.n_collab = r.n_collab + 1
"""

it = 0   
for edge in tqdm.tqdm(edges.groupby(["from"])):
    related_ids = edge[1]["to"]
    for combi in list(itertools.combinations(related_ids, 2)):
        sh1 = combi[0]
        sh2 = combi[1]
        transaction_list.append({"from":sh1,"to":sh2})
        it +=1
        if it % 1000 == 0:
            graph.run(transaction, json=transaction_list)
            transaction_list = []

graph.run(transaction, json=transaction_list)

In [ ]:
# 1) Give the IDs of the top 10 streamers that have at least one collaboration.
# Nous avons considéré le nombre de views étant plus fort
query = """MATCH (video:Streamer)
WHERE exists(video.id_)
RETURN video
ORDER BY video.views DESC
LIMIT 10"""
data = graph.run(query).to_data_frame()
list(data["video"])

# 2) create a density of node degrees.Do you observe a power scale law? what does it mean?
import seaborn as sns
import pandas as pd
import seaborn as sns
query =  """
MATCH (p)
WITH p, size((p)-[:RELATION]->()) as degree
RETURN degree;
"""
data = graph.run(query).to_data_frame()
ax = sns.distplot(data["degree"],hist=False)
ax.title.set_text("Density of degrees")

#Strong density that decreases very quickly which means that the relation tightness among the activity nodes

# 3) How many streamers has the tag "mature"?
query = """ MATCH (video:Streamer) WHERE video.mature=true with count(video) as n RETURN n """
graph.run(query)

# 4) Is the collaboration correlated to the "mature" tags?
query = """MATCH (video:Streamer)-[r:RELATION]->(n_collab)  where video.mature = TRUE  RETURN video.mature """
graph.run(query)

# 5) Do partners collaborate more than non partners ?
#les personnes qui sont des partenaires, ils vont prendre la valeur True:
query="""
MATCH (video:Streamer)
WHERE video.partner=true
with count(video) as n
RETURN n
"""
graph.run(query)

query1="""MATCH (video:Streamer) WHERE video.partner=false with count(video) as n RETURN n """
graph.run(query1)
